# This notebook reproduces all the examples in Sven Vergoolaege's paper "Counting Affine Calculator and Applications."

In [1]:
%pylab inline
# Load islpy and islplot
from islpy import *
from islplot.plotter import *

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Counting Affine Calculator and Applications
# Figure 1:
#    for (i = 0; i < N; ++i)
# S1:  t[i] = f(a[i])
#    for (i = 0; i < N; ++i)
# S2:  b[i] = g(t[N-i-1])
#
# Section 2. Syntax
idom = UnionSet("[n] -> {S1[i] : 0 <= i <= n; S2[i]: 0 <= i <= n};")
print("Iteration Domain: ",type(idom), idom)

Iteration Domain:  <class 'islpy._isl.UnionSet'> [n] -> { S1[i] : 0 <= i <= n; S2[i] : 0 <= i <= n }


In [3]:
read_acc = UnionMap("[n]-> {S1[i] -> a[i]; S2[i] -> t[n-i-1]}")
print("Read access relations: ", type(read_acc), read_acc)

Read access relations:  <class 'islpy._isl.UnionMap'> [n] -> { S1[i] -> a[i]; S2[i] -> t[-1 + n - i] }


In [4]:
write_acc = UnionMap("{S1[i] -> t[i]; S2[i]->b[i]}")
print("Writer access relations: ", type(write_acc), write_acc)

Writer access relations:  <class 'islpy._isl.UnionMap'> { S2[i] -> b[i]; S1[i] -> t[i] }


In [5]:
W = write_acc.intersect_domain(idom)
print(type(W), W)

<class 'islpy._isl.UnionMap'> [n] -> { S2[i] -> b[i] : 0 <= i <= n; S1[i] -> t[i] : 0 <= i <= n }


In [6]:
R = read_acc.intersect_domain(idom)
print(type(R), R)

<class 'islpy._isl.UnionMap'> [n] -> { S2[i] -> t[-1 + n - i] : 0 <= i <= n; S1[i] -> a[i] : 0 <= i <= n }


In [7]:
rmR = R.range_map() # range_map R
print(type(rmR), rmR)

<class 'islpy._isl.UnionMap'> [n] -> { [S2[i] -> t[-1 + n - i]] -> t[-1 + n - i] : 0 <= i <= n; [S1[i] -> a[i]] -> a[i] : 0 <= i <= n }


In [8]:
# Section 3. BASIC OPERATIONS
S = UnionMap("{S1[i]->[0,i];S2[i]->[1,i]}") #schedule.
R.compute_flow(W, W, S)

(0,
 UnionMap("[n] -> { S1[i] -> S2[i' = -1 + n - i] : 0 <= i < n }"),
 UnionMap("[n] -> {  }"),
 UnionMap("[n] -> { S2[i = n] -> t[-1] : n >= 0; S1[i] -> a[i] : 0 <= i <= n }"),
 UnionMap("[n] -> {  }"))

In [9]:
T = R.compute_flow(W,W,S)
DR = T[1] # dependence relation 
print(type(DR),DR)

<class 'islpy._isl.UnionMap'> [n] -> { S1[i] -> S2[i' = -1 + n - i] : 0 <= i < n }


In [10]:
# deltas(S^-1.Dep.S);
invS = S.reverse()
print("Schedule: ", S)
print("S^-1 = ", invS)
print("Dependence Relation:", DR)
DepDist = invS.apply_range(DR).apply_range(S)
print(type(DepDist), DepDist)
DepDist.deltas().dump()

Schedule:  { S2[i] -> [1, i]; S1[i] -> [0, i] }
S^-1 =  { [1, i1] -> S2[i = i1]; [0, i1] -> S1[i = i1] }
Dependence Relation: [n] -> { S1[i] -> S2[i' = -1 + n - i] : 0 <= i < n }
<class 'islpy._isl.UnionMap'> [n] -> { [0, i1] -> [1, -1 + n - i1] : 0 <= i1 < n }


[n] -> { [i0, i1] : exists (e0 = floor((1 + n + i1)/2): i0 = 1 and 2e0 = 1 + n + i1 and i1 < n and i1 > -n) }


In [11]:
# codegen(S * D)
D = idom
print("Schedule: ", S)
print("Iteration Domain: ", D)
out_schedule0 = S.intersect_domain(D)
print(type(out_schedule0), out_schedule0)

Schedule:  { S2[i] -> [1, i]; S1[i] -> [0, i] }
Iteration Domain:  [n] -> { S1[i] : 0 <= i <= n; S2[i] : 0 <= i <= n }
<class 'islpy._isl.UnionMap'> [n] -> { S2[i] -> [1, i] : 0 <= i <= n; S1[i] -> [0, i] : 0 <= i <= n }


In [12]:
# These print utility functions I copied from Tobias Grosser's talk:
# https://www.youtube.com/watch?v=mIBUY20d8c8
def printAST(ast):
    p = Printer.to_str(ast.get_ctx())
    p = p.set_output_format(format.C)
    p = p.print_ast_node(ast)
    print(p.get_str())
    
def printSchedule(schedule):
    p = Printer.to_str(schedule.get_ctx())
    p = p.set_yaml_style(yaml_style.BLOCK)
    p = p.print_schedule(schedule)
    print(p.get_str())
    
def printNode(node):
    p = Printer.to_str(node.get_ctx())
    p = p.set_yaml_style(yaml_style.BLOCK)
    p = p.print_schedule_node(node)
    print(p.get_str())
    
def p(obj):
    if (obj.__class__ == Schedule):
        printSchedule(obj)
    if (obj.__class__ == ScheduleNode):
        printNode(obj)
    if (obj.__class__ == AstNode):
        printAST(obj)

def printC(schedule):
    astbuild = AstBuild.from_context(Set("{:}"))
    ast = astbuild.node_from_schedule(schedule)
    p(ast)
    
def printCMap(schedule):
    astbuild = AstBuild.from_context(Set("{:}"))
    ast = astbuild.node_from_schedule_map(schedule)
    p(ast)

In [13]:
printCMap(out_schedule0)

{
  for (int c1 = 0; c1 <= n; c1 += 1)
    S1(c1);
  for (int c1 = 0; c1 <= n; c1 += 1)
    S2(c1);
}



In [14]:
S2 = UnionMap("[N]->{S1[i]->[i,0];S2[i]->[N-i-1,1]}") # Schedule - fuse S1 and S2
out_schedule1 = S2.intersect_domain(D)
printCMap(out_schedule1)

{
  for (int c0 = -n + N - 1; c0 < min(0, N); c0 += 1)
    S2(N - c0 - 1);
  for (int c0 = 0; c0 <= n; c0 += 1) {
    S1(c0);
    if (N >= c0 + 1 && n + c0 + 1 >= N)
      S2(N - c0 - 1);
  }
  for (int c0 = max(n + 1, -n + N - 1); c0 < N; c0 += 1)
    S2(N - c0 - 1);
}



In [15]:
# Section 4. Transitive Closures

# Figure 2: Flip-floop example
# double x[2][10];
# int old = 0, new = 1;
# int i,t;
# for (t = 0; t < 1000; t++) {
#   for (i = 0; i < 10; i++)
#     x[new][i] = g(x[old][i]);
#   new = (new + 1) % 2;
#   old = (old + 1) % 2;
# }

T = Map("{[new, old] -> [(new + 1) % 2, (old + 1) % 2]}")
[TC, isExact] = T.transitive_closure()
print(TC, " is exact? ", isExact)
TC.dump()

{ [new, old] -> [o0, o1] : (new + old + o0 + o1) mod 2 = 0 and 0 <= o0 <= 1 and 0 <= o1 <= 1 }  is exact?  True


{ [new, old] -> [o0, o1] : exists (e0 = floor((new + old + o0 + o1)/2): 2e0 = new + old + o0 + o1 and o0 >= 0 and o0 <= 1 and o1 >= 0 and o1 <= 1) }


In [16]:
# { [i0, i1] : exists (e0 = [(-1 - i0 + i1)/2]: 2e0= -1 - i0 + i1 and i0 >= 0 and i0 <= 1 and i1>= 0 and i1 <= 1) }
initial_state = Set("{[0,1]}")
print("INTERSECT with initial states, thenSIMPLIFY using affine hull:")
TC.intersect_domain(initial_state).affine_hull()


INTERSECT with initial states, thenSIMPLIFY using affine hull:


BasicMap("{ [new, old] -> [o0, o1] : new = 0 and old = 1 and o1 = 1 - o0 }")

In [17]:
# Figure 3. Example from [10, page 35]
#
# i = 0, j = 0
# while (i <= 100) {
#   if (A[i] <= A[j]) {
#     i = i + 2;
#     j = j + 1;
#   } else {
#     i = i + 4;
#   }
# }
#
# As a second example, reproduced in Figure 3. In this case, we consider three program points:
# at the start (zero), before the loop and after the loop (two). The transitiions between these points can
# be described as:

T = UnionMap("{zero[i,j]->one[0,0]; one[i,j] -> one[i+4,j]: i <=100; one[i,j] -> one[i+2, j+1]: i <= 100; one[i,j]->two[i,j]: i > 100}")
T.dump()
init_state = Set("{zero[i,j]}")
[TC, isExact] = T.transitive_closure()
print("TRANSITIVE CLOSURE:")
TC.dump()

#print("(T^+)(Init) + Init")
TC.intersect_domain(init_state).range().union(init_state)


TRANSITIVE CLOSURE:


{ zero[i, j] -> one[o0, o1] : o0 = 0 and o1 = 0; one[i, j] -> two[o0, o1] : o0 = i and o1 = j and i >= 101; one[i, j] -> one[o0, o1] : (o0 = 4 + i and o1 = j and i <= 100) or (o0 = 2 + i and o1 = 1 + j and i <= 100) }
{ zero[i, j] -> one[o0, o1] : (2o1 = o0 and o0 >= 2 and o0 <= 102) or (o0 = 0 and o1 = 0) or (exists (e0 = floor((o0)/4): o1 = 0 and 4e0 = o0 and o0 >= 4 and o0 <= 104)) or (exists (e0 = floor((o0)/2), e1 = floor((o0 + 2o1)/4): 2e0 = o0 and 4e1 = o0 + 2o1 and o0 <= 104 and o1 > 0 and 2o1 <= -4 + o0)); one[i, j] -> two[o0, o1] : (o0 = i and o1 = j and i >= 101) or (2o1 = -i + 2j + o0 and o0 >= 2 + i and o0 >= 101 and o0 <= 102) or (exists (e0 = floor((-i + o0)/4): o1 = j and 4e0 = -i + o0 and o0 >= 4 + i and o0 >= 101 and o0 <= 104)) or (exists (e0 = floor((i + o0)/2), e1 = floor((-i + 2j + o0 + 2o1)/4): 2e0 = i + o0 and 4e1 = -i + 2j + o0 + 2o1 and o0 >= 101 and o0 <= 104 and o1 > j and 2o1 <= -4 - i + 2j + o0)); zero[i, j] -> two[o0, o1] : (2o1 = o0 and o0 >= 101 and o0 

UnionSet("{ one[i0, i1] : i0 <= 104 and ((2i1 = i0 and 2 <= i0 <= 102) or ((i0) mod 2 = 0 and (i0 + 2i1) mod 4 = 0 and i1 > 0 and 2i1 <= -4 + i0)); one[0, 0]; one[i0, 0] : (i0) mod 4 = 0 and 4 <= i0 <= 104; two[i0, i1] : 101 <= i0 <= 104 and ((2i1 = i0 and i0 <= 102) or ((i0) mod 2 = 0 and (i0 + 2i1) mod 4 = 0 and i1 > 0 and 2i1 <= -4 + i0)); two[i0, 0] : (i0) mod 4 = 0 and 101 <= i0 <= 104; zero[i, j] }")